## Génération de matrice aléatoires

##### Comparaison des méthodes de génération de matrices triangulaires

L'objectif de cette expérience est de comparer deux approches pour générer des matrices triangulaires supérieures avec une diagonale de ±1, qui peuvent être utilisées pour construire des matrices unimodulaires.

- **Méthode 1** : Remplissage aléatoire complet de la matrice, suivi d'un ajustement pour imposer la structure triangulaire.
- **Méthode 2** : Génération directe des coefficients nécessaires aux positions pertinentes.

Bien que la Méthode 1 génère beaucoup plus de valeurs aléatoires, elle est paradoxalement plus rapide que la Méthode 2. Cela s'explique par l'efficacité des fonctions compilées utilisées pour générer des matrices aléatoires complètes, ce qui compense les ajustements supplémentaires. En revanche, la Méthode 2, bien qu'intuitive, nécessite des boucles interprétées en Python, ralentissant l'exécution.

Les résultats expérimentaux illustrent ce phénomène, avec un temps moyen significativement plus faible pour la Méthode 1.

In [1]:
import time

# Méthode 1 : Remplir la matrice aléatoirement puis ajuster les zéros et la diagonale
def generate_triangular_method1(n, l):
    U1 = MatrixSpace(ZZ, n, n).random_element(x=-(l*10), y=(l*10) + 1, distribution='uniform')
    U2 = MatrixSpace(ZZ, n, n).random_element(x=-(l*10), y=(l*10) + 1, distribution='uniform')
    
    for i in range(n):
        U1[i, i], U2[i, i] = choice([-1, 1]), choice([-1, 1])
        for j in range(i):
            U1[i, j], U2[i, j] = 0, 0
    return U1, U2

# Méthode 2 : Générer directement des valeurs au bon endroit pour une matrice triangulaire
def generate_triangular_method2(n, l):
    U1 = MatrixSpace(ZZ, n, n)(0)
    U2 = MatrixSpace(ZZ, n, n)(0)
    
    for i in range(n):
        U1[i, i], U2[i, i] = choice([-1, 1]), choice([-1, 1])
        for j in range(i + 1, n):
            U1[j, i], U2[j, i] = randint(-(l*10), (l*10)), randint(-(l*10), (l*10))
    return U1, U2

def compare_triangular_generation(n, l, repetitions=100):
    
    start_time = time.time()
    for _ in range(repetitions):
        generate_triangular_method1(n, l)
    time_method1 = (time.time() - start_time) / repetitions

    start_time = time.time()
    for _ in range(repetitions):
        generate_triangular_method2(n, l)
    time_method2 = (time.time() - start_time) / repetitions

    print(f"Temps moyen pour la Méthode 1 : {time_method1:.6f} secondes")
    print(f"Temps moyen pour la Méthode 2 : {time_method2:.6f} secondes")

In [2]:
compare_triangular_generation(n=200, l=4, repetitions=100)

Temps moyen pour la Méthode 1 : 0.025572 secondes
Temps moyen pour la Méthode 2 : 0.176002 secondes


## Génération de matrices unimodulaires

Dans cette section, nous explorons plusieurs méthodes pour générer des matrices unimodulaires (matrices entières inversibles avec un déterminant de ±1).

- **Méthode 1** : À partir de la matrice identité, on applique des opérations élémentaires pour préserver le déterminant.
- **Méthode 2** : Similaire à la première, mais en utilisant les fonctions matricielles optimisées de SageMath.
- **Méthode 3** : Produit de deux matrices triangulaires supérieures avec diagonale de ±1, suivi d'une transposition pour garantir l'unimodularité.

Nous comparons également leurs performances pour comprendre les compromis entre aléa, efficacité, et structure.

In [3]:
import time

# Méthode 1 : Génère une matrice unimodulaire en modifiant la matrice identité
def generate_unimodular_matrix1(n, l = 4, U = None, nb_passages = 10):
    
    # Initialise à une matrice identité ou utilise une matrice donnée (vérifiée comme unimodulaire)
    if U is None:
        U = [[1 if i == j else 0 for j in range(n)] for i in range(n)]
    else :
        if not (U.is_invertible()):
            raise ValueError("Erreur : la matrice générée n'est pas unimodulaire !")
        U = [[int(U[i, j]) for j in range(n)] for i in range(n)]
    
    # Applique des opérations élémentaires pour modifier la matrice tout en conservant le déterminant
    for _ in range(n * nb_passages): 
        i, j = randint(0, n - 1), randint(0, n - 1)
        
        if i != j:
            coef = randint(-(l*10), l*10 + 1)
            for k in range(n):
                U[j][k] += coef * U[i][k]
        
            if randint(0, 1):
                U[i], U[j] = U[j], U[i]
        
        if randint(0, 1):
            for k in range(n):
                U[i][k] *= -1

    U = Matrix(ZZ, U)

    # Convertit en matrice Sage et vérifie l'unimodularité
    if not (U.is_invertible()):
        raise ValueError("Erreur : la matrice générée n'est pas unimodulaire !")
    return U

# Méthode 2 : Utilise les fonctions matricielles de SageMath pour générer une matrice unimodulaire
def generate_unimodular_matrix2(n, l = 4, U = None, nb_passages = 10):
    
    # Initialise à une matrice identité ou utilise une matrice donnée
    if U is None:
        U = identity_matrix(n)
    else :
        if not (U.is_invertible()):
            raise ValueError("Erreur : la matrice générée n'est pas unimodulaire !")
    
    # Applique des transformations élémentaires directement sur la matrice Sage
    for _ in range(n * nb_passages):  
        i, j = randint(0, n - 1), randint(0, n - 1)
        if i != j:
            coef = randint(-(l*10), l*10 + 1)
            for k in range(n):
                U[j,k] += coef * U[i,k]
        
            if randint(0, 1):
                U.swap_rows(i, j)
        
        if randint(0, 1):
            U[i] *= -1
            
    # Vérifie l'unimodularité de la matrice
    if abs(U.determinant()) != 1:
        raise ValueError("Erreur : la matrice générée n'est pas unimodulaire !")
    
    return U

# Méthode 3 : Produit de deux matrices triangulaires pour générer une matrice unimodulaire
def generate_unimodular_matrix3(n, l = 4):
    
    U1 = MatrixSpace(ZZ, n,n).random_element(x = -(l*10), y = (l*10) + 1 , distribution = 'uniform')
    U2 = MatrixSpace(ZZ, n,n).random_element(x = -(l*10), y = (l*10) + 1 , distribution = 'uniform')
    
    for i in range(n):
        U1[i,i], U2[i,i] = choice([-1,1]),choice([-1,1])
        for j in range(i):
            U1[i,j], U2[i,j] = 0, 0
    
    # Génère deux matrices triangulaires supérieures aléatoires avec diagonale ±1
    U = U1 * U2.transpose()
    if not (U.is_invertible()):
        raise ValueError("Erreur : la matrice générée n'est pas unimodulaire !")
    return U

def test_keygen_performance(n, repetitions=10):
    start_time = time.time()
    for _ in range(repetitions):
        generate_unimodular_matrix1(n)
    time_1 = (time.time() - start_time) / repetitions

    start_time = time.time()
    for _ in range(repetitions):
        generate_unimodular_matrix2(n)
    time_2 = (time.time() - start_time) / repetitions
    
    start_time = time.time()
    for _ in range(repetitions):
        generate_unimodular_matrix3(n)
    time_3 = (time.time() - start_time) / repetitions

    print(f"Temps moyen pour generate_unimodular_matrix1 : {time_1:.6f} secondes")
    print(f"Temps moyen pour generate_unimodular_matrix2 : {time_2:.6f} secondes")
    print(f"Temps moyen pour generate_unimodular_matrix3 : {time_3:.6f} secondes")

In [4]:
test_keygen_performance(n=200, repetitions=20)

Temps moyen pour generate_unimodular_matrix1 : 1.790295 secondes
Temps moyen pour generate_unimodular_matrix2 : 1.906735 secondes
Temps moyen pour generate_unimodular_matrix3 : 0.264814 secondes


#### Analyse des performances

Au vu des résultats, la troisième méthode (produit de deux matrices triangulaires) est la plus rapide pour générer une matrice unimodulaire. Cependant, son aléa reste limité : par exemple, la dernière ligne de la matrice générée se termine souvent par ±1.

Nous allons appliquer des transformations élémentaires à la matrice issue de la méthode 3 pour augmenter l'aléa tout en conservant l'unimodularité.

In [5]:
def KeyGen(n, l = 4, debug = False):
    B = MatrixSpace(ZZ, n,n).random_element(x = -l, y = l + 1 , distribution = 'uniform')
    while (B.hermite_form().column(n-1) == vector(ZZ, [0] * n)):
        B = MatrixSpace(ZZ, n,n).random_element(x = -l, y = l + 1 , distribution = 'uniform')
        
    U = generate_unimodular_matrix3(n, 100) # l = 100 pour illustrer la problématique
        
    B_prime = B * U
    
    if(debug):
        print("Matrice B (clé publique) :")
        print(B)
        print("\nMatrice U (unimodulaire) :")
        print(U)
        print("\nLigne problématique :")
        print(U[n-1])
        print("\nMatrice B' (clé privée) :")
        print(B_prime)

    return B, B_prime

In [6]:
Bpr, Bpb = KeyGen(10, debug=1)

Matrice B (clé publique) :
[ 3 -4 -4 -3 -4  2  1 -4  1  1]
[-3 -3  1 -4 -4 -3  3  4  1 -1]
[ 4  0  0  0  1  4  3 -3  0  4]
[ 3 -1 -3  2  1  4 -2 -1 -2 -1]
[-2  4  3  2  0 -2  3  1 -4  3]
[-4  1  4 -3 -1 -1 -2 -4 -2  4]
[-4  1  4  2 -3 -3  4  0 -4  3]
[ 4  2 -1 -2  4 -1  2  3 -4  3]
[ 0  0  2 -1  3 -4  4  2  0  0]
[ 4  0  0  4  4 -3 -3 -3  4 -4]

Matrice U (unimodulaire) :
[  179366  -801668   487576   876344   675449   118582  -805030  -167293    36886       71]
[-1601539   382856 -1804294   912929  1042201    92465  -821552   441633   316129      604]
[ -292012   862729 -1470624 -1082142 -1123935  -931649   710405   -32556  -388243     -743]
[-1083641  -624199    95272   769837  1644455   999221  -545752   266949   430857      824]
[  604219  -992448     2997  1001546   111040  -881273  -489670 -1121595  -517613     -988]
[  308040  -303521   715522      826   104403   248393   351130  -230153   191986      368]
[ -133776   -55721   607522  -342838   254524   685676   535213   170979 

Nous allons vérifier que les transformations élémentaires appliquées sur la matrice générée par la méthode 3 ne sont pas trop consommatrices en ressources. Ensuite, nous mesurerons les temps moyens pour deux méthodes utilisant ces transformations et comparerons leurs performances.

In [7]:
def KeyGen(n, l = 4, debug = False):
    B = MatrixSpace(ZZ, n,n).random_element(x = -l, y = l + 1 , distribution = 'uniform')
    while (B.hermite_form().column(n-1) == vector(ZZ, [0] * n)):
        B = MatrixSpace(ZZ, n,n).random_element(x = -l, y = l + 1 , distribution = 'uniform')
        
    U = generate_unimodular_matrix1(n, l, generate_unimodular_matrix3(n), 4)
        
    B_prime = B * U
    
    if(debug):
        print("Matrice B (clé publique) :")
        print(B)
        print("\nMatrice U (unimodulaire) :")
        print(U)
        print("\nLigne anciennement problématique :")
        print(U[n-1])
        print("\nMatrice B' (clé privée) :")
        print(B_prime)

    return B, B_prime

In [8]:
Bpr, Bpb = KeyGen(8, debug=1)

Matrice B (clé publique) :
[-4 -2  0  4 -3 -2  4  3]
[ 1 -3 -3  4 -3  0 -3  2]
[-3  3 -2 -1 -2  2  3 -4]
[ 3  1  1 -3 -1 -3 -3  3]
[ 4 -1  2  2 -1 -3  4  1]
[-4 -2  3 -1  1  1  2 -3]
[ 3  2 -4 -3  2 -3 -3  0]
[-1  2  3 -3 -3  0  2 -4]

Matrice U (unimodulaire) :
[ 318078355068  598849049767    3704168738 -200777456828  363411517085 -198246375077   -3525601126   -2128782816]
[   -837264860   -1573799951     -12143172     529080415    -957390111     521914984       9199293       5639956]
[   -186322878    -341474066     -10784746     119533490    -215737513     116503239       1780134       1377208]
[   2971513402    5445849628     172014367   -1906342673    3440632230   -1858012583     -28388344     -21964591]
[       253858       -113419        564202       -296600        476081       -176966         16098        -10214]
[   9646849952   18162210736     112344390   -6089286338   11021739056   -6012521842    -106926229     -64562897]
[     -4264633       1824213      -9459265       4989

In [9]:
def test_keygen_performance(n, l = 4, repetitions = 10):
    start_time = time.time()
    for _ in range(repetitions):
        generate_unimodular_matrix1(n, l, generate_unimodular_matrix3(n), 4)
    time_1 = (time.time() - start_time) / repetitions
    
    for _ in range(repetitions):
        generate_unimodular_matrix2(n, l, generate_unimodular_matrix3(n), 4)
    time_2 = (time.time() - start_time) / repetitions

    print(f"Temps moyen pour generate_unimodular_matrix1 sur generate_unimodular_matrix3 : {time_1:.6f} secondes")
    print(f"Temps moyen pour generate_unimodular_matrix2 sur generate_unimodular_matrix3 : {time_2:.6f} secondes")

In [10]:
test_keygen_performance(n=200, repetitions=20)

Temps moyen pour generate_unimodular_matrix1 sur generate_unimodular_matrix3 : 1.087504 secondes
Temps moyen pour generate_unimodular_matrix2 sur generate_unimodular_matrix3 : 2.298526 secondes


Il existe également une approche supplémentaire : générer une matrice identité, modifier les coefficients d'une ligne ou d'une colonne aléatoire, et répéter cette opération plusieurs fois pour améliorer l'aléa. En multipliant ces matrices entre elles, nous obtenons une matrice unimodulaire de meilleure qualité. Nous allons coder cette méthode et évaluer son temps d'exécution pour générer 2n matrices.

In [11]:
"""
Génère une matrice unimodulaire aléatoire.

Paramètres :
- n (int) : Taille de la matrice carrée (n x n).

Retour :
- Matrix : Une matrice unimodulaire de dimensions n x n (déterminant égal à ±1).
"""
def generate_unimodular_matrix(n):
    Ures = matrix.identity(n)
    dist = GeneralDiscreteDistribution([1/7, 5/7, 1/7])
    
    for i in range(2 * n):
        U = matrix.identity(n)
        ind = randint(0,n-1)
        
        if randint(0,1): # Modifie une ligne ou une colonne avec 50% de chances
            U[ind] = [dist.get_random_element() - 1 for _ in range(n)]
            U[ind, ind] = 1
        else:
            for i in range(n):
                U[i, ind] = dist.get_random_element() - 1
            U[ind, ind] = 1
        Ures *= U
    return Ures

In [12]:
def generate_and_multiply(n, repetitions=100):
    total_time = 0

    for _ in range(repetitions):
        start_time = time.time()
        U = generate_unimodular_matrix(n)
        total_time += time.time() - start_time

    time1 = total_time / repetitions

    print(f"Temps total pour générer et multiplier 2*n matrices unimodulaires : {time1:.6f} secondes")
    #print("\nMatrice U (unimodulaire) :")
    #print(U)
    
generate_and_multiply(n=200, repetitions=10)

Temps total pour générer et multiplier 2*n matrices unimodulaires : 3.245453 secondes


Avec un temps d'environ 1 secondes pour une taille de 200, cette méthode est significativement plus lente que les autres approches testées. Bien qu'elle soit fonctionnelle, elle n'est pas optimale pour une implémentation pratique ou à grande échelle.

## Comparaison des générations de matrice identité

In [13]:
def test_identity_performance(n, repetitions=400):
    T = matrix.identity(n)
    start_time = time.time()
    for _ in range(repetitions):
        T *= identity_matrix(ZZ, n,n)
    time_1 = (time.time() - start_time) / repetitions

    start_time = time.time()
    for _ in range(repetitions):
        T *= matrix.identity(n)
    time_2 = (time.time() - start_time) / repetitions
    

    print(f"Temps moyen pour identity_matrix : {time_1:.6f} secondes")
    print(f"Temps moyen pour matrix.identity : {time_2:.6f} secondes")
test_identity_performance(200)

Temps moyen pour identity_matrix : 0.020641 secondes
Temps moyen pour matrix.identity : 0.006386 secondes


On utilisera donc dans le reste du TP matrix.identity(n). Ce résultat est d'autant plus significatif que l'on peut trouver des résultats pouvant aller jusqu'à un temps 30 fois plus long dans certains cas.

## Tests d'inversion
Voici un test comparant deux méthodes pour vérifier l'inversibilité d'une matrice unimodulaire : la méthode is_invertible(), qui utilise la fonction intégrée de SageMath, et la méthode basée sur le calcul explicite du déterminant, abs(U.determinant()) != 1. Les deux approches présentent des temps d'exécution similaires, bien que is_invertible() soit légèrement plus lente. Cependant, les deux méthodes restent suffisamment rapides pour une utilisation dans des implémentations réelles.

In [14]:
import time

def test_invertibility_check(n, repetitions=10):
    # Génération d'une grande matice unimodulaire
    U = generate_unimodular_matrix1(n,4)

    # Test avec is_invertible()
    start_time = time.time()
    for _ in range(repetitions):
        U.is_invertible()
    time_invertible = (time.time() - start_time) / repetitions

    # Test avec abs(U.determinant()) != 1
    start_time = time.time()
    for _ in range(repetitions):
        abs(U.determinant()) != 1
    time_determinant = (time.time() - start_time) / repetitions

    print(f"Temps moyen pour .is_invertible(): {time_invertible:.6f} secondes")
    print(f"Temps moyen pour abs(U.determinant()) != 1: {time_determinant:.6f} secondes")

test_invertibility_check(n=200, repetitions=100)

Temps moyen pour .is_invertible(): 0.000001 secondes
Temps moyen pour abs(U.determinant()) != 1: 0.000000 secondes


# Comparaison des méthodes de génération exhaustive de vecteurs d'erreurs

In [15]:
import time
from itertools import product

def test_vector_generation_performance(dim, r, repetitions=10):
    print(f"Test pour dim={dim}, r={r}, répétitions={repetitions}\n")

    # Temps pour la méthode generate_vectors (génération complète)
    full_generation_times = []
    for _ in range(repetitions):
        start_time = time.time()
        vectors = generate_vectors(dim, r)
        full_generation_times.append(time.time() - start_time)

    # Temps pour la méthode generate_vectors_iterative (génération itérative)
    iterative_full_generation_times = []
    iterative_first_generation_times = []
    for _ in range(repetitions):
        # Temps pour obtenir le premier vecteur
        start_time = time.time()
        gen = generate_vectors_iterative(dim, r)
        _ = next(gen, None)
        iterative_first_generation_times.append(time.time() - start_time)

        # Temps total pour parcourir tous les vecteurs
        start_time = time.time()
        gen = generate_vectors_iterative(dim, r)
        for _ in gen:
            pass
        iterative_full_generation_times.append(time.time() - start_time)

    avg_full_generation_time = sum(full_generation_times) / repetitions
    avg_iterative_full_time = sum(iterative_full_generation_times) / repetitions
    avg_iterative_first_time = sum(iterative_first_generation_times) / repetitions
    
    print("generate_vectors :")
    print(f"- Temps moyen (génération complète) : {avg_full_generation_time:.6f} secondes")
    print(f"- Temps moyen (premier vecteur)     : {avg_full_generation_time:.6f} secondes\n")
    
    print("generate_vectors_iterative :")
    print(f"- Temps moyen (génération complète) : {avg_iterative_full_time:.6f} secondes")
    print(f"- Temps moyen (premier vecteur)     : {avg_iterative_first_time:.6f} secondes")

# Fonctions à comparer
def generate_vectors(dim, r):
    all_vectors = [vector(v) for v in product([0, -1, 1], repeat=dim)]
    valid_vectors = [v for v in all_vectors if v.norm() < r]
    return valid_vectors

def generate_vectors_iterative(dim, r):
    for v in product([0, -1, 1], repeat=dim):
        v = vector(v)
        if v.norm() < r:
            yield v

# Test des performances
test_vector_generation_performance(dim=6, r=3, repetitions=10)

Test pour dim=6, r=3, répétitions=10

generate_vectors :
- Temps moyen (génération complète) : 0.889914 secondes
- Temps moyen (premier vecteur)     : 0.889914 secondes

generate_vectors_iterative :
- Temps moyen (génération complète) : 0.786026 secondes
- Temps moyen (premier vecteur)     : 0.000156 secondes


Bien que la méthode itérative soit légèrement plus lente pour la génération complète des vecteurs, elle offre un avantage clé dans les attaques par force brute : un accès quasi-instantané au premier résultat (0,000048 s contre 0,167953 s). Cela permet de tester rapidement des solutions sans attendre la génération complète, optimisant ainsi le temps de recherche. De plus, elle réduit la consommation mémoire en générant les vecteurs au fur et à mesure, ce qui est essentiel pour traiter de grands espaces de solutions.

### Comparaison des fonctions d'arrondi

In [16]:
test = [-2.5,-1.5,-0.5,0.5,1.5,2.5,3.5]
print(vector([round(i) for i in test]))
print(vector(test).apply_map(lambda x: math.ceil(x) if abs(x % 1) == 0.5 else round(x)))

(-3, -2, -1, 1, 2, 3, 4)
(-2, -1, 0, 1, 2, 3, 4)
